
<div align="center"><img width="275" height="50" src="http://zillow.mediaroom.com/image/Zillow_Wordmark_Blue_RGB.jpg" /> </div> 

<div align="center"> <h1>Cluster Project</h1> 
  <h6> by John Grinstead & David Berchelmann -- April 7, 2021 </h6> </div>
  
  ------------------------------------------------

<div align="center"><img width="800" height="50" src="https://www.zillowstatic.com/s3/homepage/static/Buy_a_home.png" /> </div>



-------

<h1> Welcome! </h1>

The following jupyter notebook will take you through my regression project focusing on Zillow. The dataset comes from a SQL database and can also be accessed via Kaggle.com. 

All of the files and notebooks for this project can be accessed via the github repostiory located at --> https://github.com/davidb-and-john/clustering-project



----

<h1> Executive Summary </h1>

------

<h4><b>The Problem</b></h4>

- What is driving the errors in the Zestimates?

<h4><b>The Goal</b></h4>

- Use clustering to identify what groups of features are the strongest drivers of log error.

<h4><b>The Process</b></h4>

  * Acquire the Data
  * Prepare 
  * Explore 
  * Model
  * Create Recommendations Based On Findings 
  
<h4><b>The Findings</b></h4>

    
    


-------


<h3><u>Environment Setup</u></h3>

In [1]:
# packages for data analysis & mapping
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.dates as dates
import seaborn as sns
import plotly.express as px
from datetime import date 


# Statistical Tests
import scipy.stats as stats
from math import sqrt
from scipy.stats import norm


# modeling methods
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest, f_regression, RFE 
import sklearn.preprocessing
pd.options.display.float_format = '{:20,.2f}'.format


# address warnings
import warnings
warnings.filterwarnings("ignore")

# acquire, prep, train, & explore functions
from wrangle import get_connection, new_zillow_data, get_zillow_data, clean_zillow, split, seperate_y, scale_data, split_seperate_scale 



<h4> Data Validation </h4>

 - Before the data was brought in through the acquire file, we investigated the set in SQL. Below are a few of our findings:
     - Some properties were labeled as 'single family residential' but had a unit count of more than 1
     - There were a number of properties that lacked location info (zip, lat, long, fips), these will be dropped in prep
     - Bedrooms and Bathrooms both have rows that have a value of 0. These were filled filled with the median count for each feature.
     - There were entries that had multiple transaction dates. To account for this, we filtered for the latest max date. We also did this for log error.

---
<h3><u>Acquire the Data</u></h3>

----

In [2]:
df = pd.read_csv("zillowcluster_df.csv")
print(f'Our original dataframe is coming in with {df.shape[0]} rows and {df.shape[1]} columns.')

Our original dataframe is coming in with 77413 rows and 68 columns.


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77413 entries, 0 to 77412
Data columns (total 68 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    77413 non-null  int64  
 1   typeconstructiontypeid        222 non-null    float64
 2   storytypeid                   50 non-null     float64
 3   heatingorsystemtypeid         49439 non-null  float64
 4   buildingclasstypeid           15 non-null     float64
 5   architecturalstyletypeid      206 non-null    float64
 6   airconditioningtypeid         24953 non-null  float64
 7   parcelid                      77413 non-null  int64  
 8   id                            77413 non-null  int64  
 9   basementsqft                  50 non-null     float64
 10  bathroomcnt                   77380 non-null  float64
 11  bedroomcnt                    77380 non-null  float64
 12  buildingqualitytypeid         49671 non-null  float64
 13  c

In [15]:
df.describe()

,Unnamed: 0,typeconstructiontypeid,storytypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,basementsqft,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,logerror
count,"77,413.00",222.00,50.00,"49,439.00",15.00,206.00,"24,953.00","77,413.00","77,413.00",50.00,...,"17,560.00",172.00,"77,268.00","77,379.00","77,380.00","77,378.00","77,375.00","2,886.00","77,136.00","77,413.00"
mean,"38,706.00",6.04,7.00,3.92,3.93,7.39,1.81,"13,006,697.32","1,495,761.15",679.72,...,1.43,1.00,"189,315.98","490,137.46","2,016.00","301,096.94","5,995.57",14.09,"60,496,735,236,339.62",0.02
std,"22,347.35",0.56,0.00,3.59,0.26,2.73,2.97,"3,480,712.14","861,270.67",689.70,...,0.54,0.00,"230,088.56","653,447.67",0.00,"492,599.03","7,622.89",2.19,"1,535,251,869,244.56",0.17
min,0.00,4.00,7.00,1.00,3.00,2.00,1.00,"10,711,855.00",349.00,38.00,...,1.00,1.00,44.00,"1,000.00","2,016.00",161.00,19.92,3.00,"60,371,011,101,000.00",-4.66
25%,"19,353.00",6.00,7.00,2.00,4.00,7.00,1.00,"11,538,336.00","752,413.00",273.00,...,1.00,1.00,"84,265.00","207,000.00","2,016.00","85,504.00","2,715.59",14.00,"60,373,109,005,001.75",-0.02
50%,"38,706.00",6.00,7.00,2.00,4.00,7.00,1.00,"12,530,673.00","1,498,471.00",515.00,...,1.00,1.00,"136,499.50","358,976.00","2,016.00","203,383.50","4,450.72",15.00,"60,376,032,003,008.00",0.01
75%,"58,059.00",6.00,7.00,7.00,4.00,7.00,1.00,"14,211,486.00","2,241,718.00",796.50,...,2.00,1.00,"218,787.50","569,003.00","2,016.00","366,802.25","6,927.80",15.00,"60,590,423,251,008.00",0.04
max,"77,412.00",13.00,7.00,24.00,4.00,21.00,13.00,"167,689,317.00","2,985,182.00","3,560.00",...,6.00,1.00,"11,421,790.00","49,061,236.00","2,016.00","48,952,198.00","586,639.30",99.00,"483,030,105,084,015.00",5.26


In [3]:
df.isna().sum()

Unnamed: 0                          0
typeconstructiontypeid          77191
storytypeid                     77363
heatingorsystemtypeid           27974
buildingclasstypeid             77398
architecturalstyletypeid        77207
airconditioningtypeid           52460
parcelid                            0
id                                  0
basementsqft                    77363
bathroomcnt                        33
bedroomcnt                         33
buildingqualitytypeid           27742
calculatedbathnbr                 642
decktypeid                      76799
finishedfloor1squarefeet        71390
calculatedfinishedsquarefeet      229
finishedsquarefeet12             3665
finishedsquarefeet13            77372
finishedsquarefeet15            74404
finishedsquarefeet50            71390
finishedsquarefeet6             77027
fips                               33
fireplacecnt                    69137
fullbathcnt                       642
garagecarcnt                    51939
garagetotals

-----
<h3> Initial Thoughts </h3>

This is a really messy data set that is going to require a lot of cleaning. With so many nulls, each category will be reviewed to determine if there is a way to fill those nulls with values or if they will need to be deleted all togther.

There is redundancy in columns (like those dealing with square footage) that will need to be dealt with

-----

-----

<h3>Takeaways From Acquire</h3>

-----

----

<h3>Clean/Prep the Data</h3>

----

-----
<h3> Prep Takeaways </h3>

---------

<h3><u>Data Dictionary</u></h3>



|   Feature      |  Data Type   | Description    |
| :------------- | :----------: | -----------: |
|  parcelid | float64  | Unique parcel identifier    |
| heatingorsystemtypeid    | float64| Identifier for heating type|
| airconditioningtypeid  | float64 | Identifier for ac type|
| bathroomcnt | float64 |number of bathrooms in property|
|  bedroomcnt    | float64  | number of bedrooms in property   |
| calculatedfinishedsquarefeet   | float64 | total livable square footage|
| fips    | object| Federal Information Processing Code (county code)|
| latitude | float64 | geographic coordinate that specifies the north–south position|
|  longitude  | float64   | geographic coordinate that specifies the east-west position |
| poolcnt    | float64 | has pool = 1, no pool = 0|
| roomcnt   | float64 | count of rooms in property|
| yearbuilt   | float64 | year home was built |
|  fireplaceflag  | int64   | Has fireplace = 1, no fireplace = 0     |
| taxvaluedollarcnt   | float64 | The most recent year property taxes were assessed|
| taxamount   | float64 | ad valorem tax on the value of a property.|
| logerror  | float64 | age of home as of today's date in years|
| transactiondate     | datetime64[ns] | date property was last sold|
| airconditioningdesc   | object | description of AC type|
| heatingorsystemdesc  | object| description of Heating type|
|age_of_home  | float64   | Current date - Year Built in years    |
|  age_bin  | float64 | year home was built|
|  baths_per_sqft | float64 | numbers of baths per sqft|
|  taxrate   | float64 | This is property tax / tax_assessed_value |
|  acres   | float64   | lot square footage / 43,560     |
|  acres_bin   | float64 | properties binned into groups by acreage amounts|
|  sqft_bin | float64 | properties binned into groups by square footage amounts|
|  bath_bed_ratio  | float64 | bathroomcnt/bedroomcnt|
|   la_county   | uint8  | property in LA = 1, not in LA = 0|
|  orange_county  | uint8  | property in Orange = 1, not in Orange = 0|
| ventura_county  | uint8 | property in Ventura = 1, not in ventura = 0|
















-----

<h3> Explore the Data </h3>

-----

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77413 entries, 0 to 77412
Data columns (total 68 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    77413 non-null  int64  
 1   typeconstructiontypeid        222 non-null    float64
 2   storytypeid                   50 non-null     float64
 3   heatingorsystemtypeid         49439 non-null  float64
 4   buildingclasstypeid           15 non-null     float64
 5   architecturalstyletypeid      206 non-null    float64
 6   airconditioningtypeid         24953 non-null  float64
 7   parcelid                      77413 non-null  int64  
 8   id                            77413 non-null  int64  
 9   basementsqft                  50 non-null     float64
 10  bathroomcnt                   77380 non-null  float64
 11  bedroomcnt                    77380 non-null  float64
 12  buildingqualitytypeid         49671 non-null  float64
 13  c

In [8]:
df.describe()

,Unnamed: 0,typeconstructiontypeid,storytypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,basementsqft,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,logerror
count,"77,413.00",222.00,50.00,"49,439.00",15.00,206.00,"24,953.00","77,413.00","77,413.00",50.00,...,"17,560.00",172.00,"77,268.00","77,379.00","77,380.00","77,378.00","77,375.00","2,886.00","77,136.00","77,413.00"
mean,"38,706.00",6.04,7.00,3.92,3.93,7.39,1.81,"13,006,697.32","1,495,761.15",679.72,...,1.43,1.00,"189,315.98","490,137.46","2,016.00","301,096.94","5,995.57",14.09,"60,496,735,236,339.62",0.02
std,"22,347.35",0.56,0.00,3.59,0.26,2.73,2.97,"3,480,712.14","861,270.67",689.70,...,0.54,0.00,"230,088.56","653,447.67",0.00,"492,599.03","7,622.89",2.19,"1,535,251,869,244.56",0.17
min,0.00,4.00,7.00,1.00,3.00,2.00,1.00,"10,711,855.00",349.00,38.00,...,1.00,1.00,44.00,"1,000.00","2,016.00",161.00,19.92,3.00,"60,371,011,101,000.00",-4.66
25%,"19,353.00",6.00,7.00,2.00,4.00,7.00,1.00,"11,538,336.00","752,413.00",273.00,...,1.00,1.00,"84,265.00","207,000.00","2,016.00","85,504.00","2,715.59",14.00,"60,373,109,005,001.75",-0.02
50%,"38,706.00",6.00,7.00,2.00,4.00,7.00,1.00,"12,530,673.00","1,498,471.00",515.00,...,1.00,1.00,"136,499.50","358,976.00","2,016.00","203,383.50","4,450.72",15.00,"60,376,032,003,008.00",0.01
75%,"58,059.00",6.00,7.00,7.00,4.00,7.00,1.00,"14,211,486.00","2,241,718.00",796.50,...,2.00,1.00,"218,787.50","569,003.00","2,016.00","366,802.25","6,927.80",15.00,"60,590,423,251,008.00",0.04
max,"77,412.00",13.00,7.00,24.00,4.00,21.00,13.00,"167,689,317.00","2,985,182.00","3,560.00",...,6.00,1.00,"11,421,790.00","49,061,236.00","2,016.00","48,952,198.00","586,639.30",99.00,"483,030,105,084,015.00",5.26


-----

<h3> Model the Data </h3>

-----

-------

<h3>Conclusions & Thoughts Moving Forward</h3>

-----